In [1]:
#!/usr/bin/env python
# coding: utf-8

import glob
import numpy as np
import os
import pandas as pd
import pybedtools as pbt
import subprocess
import sys

sys.path.append(os.getcwd())
import UKBB_traits

sys.path.append("/dors/capra_lab/users/fongsl/tools/py_/")
import config_readwrite as crw

import split_filename

name = "/data/hodges_lab/ATAC-STARR_B-cells/bin_human-evolution/config"
config, configfile_name = crw.read_config(name)
PATH =config["UKBB"]["path"] 

LD_PY = config["GWAS_CATALOG"]['bin_ld_expand']
ANC = config["GWAS_CATALOG"]["ANC"]
NOMINAL_P = float(config["GWAS_CATALOG"]["NOMINAL_P"])

#hg38 6p21
HLA_BED="/data/hodges_lab/ATAC-STARR_B-cells/bin_human-evolution/GWAS_catalog/ukbb/hla.bed"

# subtract hla from traits 

In [ ]:
dl_dict = UKBB_traits.dl_dict() # second round of traits


for f in dl_dict.values():

    f=(f.strip(".bgz")).split("/")[-1]
    path, filename, sid = split_filename.split_filename(f)
    out_uniq = os.path.join(PATH, (sid + "clean_LD_exp_uniq.bed")) #intersect this
    out_nohla = os.path.join(PATH, (sid + "clean_LD_exp_uniq_nohla.bed")) # write this

    hla = pbt.BedTool(HLA_BED)
    bed = pbt.BedTool(out_uniq)

    no_hla = bed.intersect(hla, v=True).saveas(out_nohla)  # subtract 
    
    w_hla=sum(1 for lin in bed)
    wo_hla=sum(1 for lin in no_hla)
    
    print(sid, w_hla, wo_hla)

categorical-20002-both_sexes-1381 46300 24982
categorical-20002-both_sexes-1464 77191 34851
categorical-20002-both_sexes-1452 53112 41303
categorical-20002-both_sexes-1111 185347 132894
categorical-20002-both_sexes-1463 49484 29672
phecode-202.2-both_sexes 37994 25855
continuous-30000-both_sexes-irnt 563041 495837
categorical-20002-both_sexes-1387 91521 58995
continuous-30200-both_sexes-irnt 465498 421485
continuous-845-both_sexes 151361 146053
phecode-290.11-both_sexes 22850 22654
categorical-22506-both_sexes-111 17203 16759
continuous-30120-both_sexes-irnt 631201 558985
continuous-30080-both_sexes-irnt 854496 788223
continuous-30100-both_sexes-irnt 946121 876737
phecode-174-both_sexes 72256 71113

# subtract regions

In [2]:
BKGD = config["BKGD"]["merged_sharedacc_0000"]  # background to shuffle in
REGIONS_PATH = config["CIS_TRANS"]["regions_dir"]

RE = config["UKBB"]["results"]
PATH = config["UKBB"]["path"]

In [6]:
## file names to test for enrichment:

test_filenames = [

                "cis_only.bed",
                "cis+trans.bed",
                "trans_only.bed",
                "conserved_active.regions.bed",
    
                "HH-active_MM-inactive_cis-only.bed",
                "HH-active_MM-inactive_cis+trans.bed",
                "HH-active_MM-inactive_trans-only.bed",
    
                "MM-active_HH-inactive_cis-only.bed",
                "MM-active_HH-inactive_cis+trans.bed", 
                "MM-active_HH-inactive_trans-only.bed",

]
    
for f in test_filenames:
    full_bed = os.path.join(REGIONS_PATH, f)
    sid = f.split(".bed")[0]
    out_nohla = os.path.join(REGIONS_PATH, (sid + "_nohla.bed")) # write this
    
    
    hla = pbt.BedTool(HLA_BED)
    bed = pbt.BedTool(full_bed)

    no_hla = bed.intersect(hla, v=True).saveas(out_nohla)  # subtract 
    yes_hla = bed.intersect(hla)  # find overlap
    
    w_hla=sum(1 for lin in bed)
    wo_hla=sum(1 for lin in no_hla)
    dif = w_hla-wo_hla
    print("\n", sid, w_hla, wo_hla, dif,"\n")
    
    for i in yes_hla:
        print(i)


 cis_only 2058 2054 4 

chr6	32609870	32610120	chr6:32609870-32610120

chr6	32637259	32637389	chr6:32637259-32637389

chr6	33571421	33571601	chr6:33571421-33571601

chr6	33744613	33744753	chr6:33744613-33744753


 cis+trans 10176 10175 1 

chr6	33556917	33557037	chr6:33556917-33557037


 trans_only 2047 2046 1 

chr6	33789133	33789253	chr6:33789133-33789253


 conserved_active.regions 2397 2397 0 


 HH-active_MM-inactive_cis-only 1074 1072 2 

chr6	32637259	32637389	chr6:32637259-32637389

chr6	33571421	33571601	chr6:33571421-33571601


 HH-active_MM-inactive_cis+trans 5435 5435 0 


 HH-active_MM-inactive_trans-only 791 791 0 


 MM-active_HH-inactive_cis-only 984 982 2 

chr6	32609870	32610120	chr6:32609870-32610120

chr6	33744613	33744753	chr6:33744613-33744753


 MM-active_HH-inactive_cis+trans 4741 4740 1 

chr6	33556917	33557037	chr6:33556917-33557037


 MM-active_HH-inactive_trans-only 1256 1255 1 

chr6	33789133	33789253	chr6:33789133-33789253



# for peaks 

In [7]:
## file names to test for enrichment:

test_filenames = [

                "cis_only.bed",
                "cis+trans.bed",
                "trans_only.bed",
                "conserved_active.regions.bed",
    
                "HH-active_MM-inactive_cis-only.bed",
                "HH-active_MM-inactive_cis+trans.bed",
                "HH-active_MM-inactive_trans-only.bed",
    
                "MM-active_HH-inactive_cis-only.bed",
                "MM-active_HH-inactive_cis+trans.bed", 
                "MM-active_HH-inactive_trans-only.bed",

]
    
for f in test_filenames:
    
    peak_f = f"peaks-{f}"
    full_bed = os.path.join(REGIONS_PATH, peak_f)

    sid = peak_f.split(".bed")[0]
    out_nohla = os.path.join(REGIONS_PATH, (sid + "_nohla.bed")) # write this
    
    
    hla = pbt.BedTool(HLA_BED)
    bed = pbt.BedTool(full_bed)

    no_hla = bed.intersect(hla, v=True).saveas(out_nohla)  # subtract 
    
    w_hla=sum(1 for lin in bed)
    wo_hla=sum(1 for lin in no_hla)
    
    dif = w_hla-wo_hla
    print("\n", sid, w_hla, wo_hla, dif,"\n")


 peaks-cis_only 1821 1817 4 


 peaks-cis+trans 7298 7297 1 


 peaks-trans_only 1796 1795 1 


 peaks-conserved_active.regions 2045 2045 0 


 peaks-HH-active_MM-inactive_cis-only 984 982 2 


 peaks-HH-active_MM-inactive_cis+trans 4525 4525 0 


 peaks-HH-active_MM-inactive_trans-only 753 753 0 


 peaks-MM-active_HH-inactive_cis-only 890 888 2 


 peaks-MM-active_HH-inactive_cis+trans 3791 3790 1 


 peaks-MM-active_HH-inactive_trans-only 1131 1130 1 



# subtract hla frm bkgd

In [9]:
bkgd=pbt.BedTool(BKGD)
out_nohla = os.path.join(REGIONS_PATH, "bkgd_nohla.bed") # write this

no_hla = bkgd.intersect(hla, v=True).saveas(out_nohla)  # subtract 

w_hla=sum(1 for lin in bkgd)
wo_hla=sum(1 for lin in no_hla)
dif = w_hla-wo_hla
print("\nbkgd", w_hla, wo_hla, dif,"\n")



bkgd 50683 50656 27 



In [ ]:
def main(argv):

    dl_dict = UKBB_traits.dl_dict() # second round of traits


    for f in dl_dict.values():

        f=(f.strip(".bgz")).split("/")[-1]
        path, filename, sid = split_filename.split_filename(f)
        out_uniq = os.path.join(PATH, (sid + "clean_LD_exp_uniq.bed")) #intersect this
        out_nohla = os.path.join(PATH, (sid + "clean_LD_exp_uniq_nohla.bed")) # write this

        hla = pbt.BedTool(HLA_BED)
        bed = pbt.BedTool(out_uniq)

        no_hla = bed.intersect(hla, v=True).saveas(out_nohla)  # subtract 

        w_hla=sum(1 for lin in bed)
        wo_hla=sum(1 for lin in no_hla)

        print(sid, w_hla, wo_hla)
if __name__ == "__main__":
    main(sys.argv[1:])